# Alter
- Linechart mit 3 Alterskohorten pro Bachelor/Master
- Stacked Barchart aggregiert über alle Studiengänge über die Jahre


In [24]:
import pandas as pd
import plotly.express as px

# Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Erstes Blatt entfernen
sheets.pop(next(iter(sheets)))

# Dictionary zur Speicherung der Altersgruppen
alter_data = {"Jahr": []}

# Durch alle Jahre iterieren
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()  # Spaltennamen bereinigen

    # Studiengänge für Bachelor & Master automatisch erkennen
    bachelor_studiengaenge = df.filter(like="Bachelor").columns
    master_studiengaenge = df.filter(like="Master").columns

    # Gesamtanzahl der Studierenden pro Jahr für Gewichtung berechnen (nur für Variable "Alter")
    total_bachelor_students = df[
        (df["Variable"] == "Alter") & (df["Category"].str.contains("Anzahl", na=False))
    ][bachelor_studiengaenge].sum(axis=1).values[0]

    total_master_students = df[
        (df["Variable"] == "Alter") & (df["Category"].str.contains("Anzahl", na=False))
    ][master_studiengaenge].sum(axis=1).values[0]

    # **Gesamtanzahl aller Studierenden berechnen**
    total_students = total_bachelor_students + total_master_students

    # Altersgruppen extrahieren
    altersgruppen = [
        "unter 18 Jahre (%)",
        "18 bis 20 Jahre (%)",
        "21 bis 23 Jahre (%)",
        "24 bis 26 Jahre (%)",
        "27 bis 29 Jahre (%)",
        "30 bis 32 Jahre (%)",
        "33 bis 35 Jahre (%)",
        "älter als 35 Jahre (%)"
    ]

    if jahr not in alter_data["Jahr"]:
        alter_data["Jahr"].append(jahr)

    for category in altersgruppen:
        row = df[
            (df["Variable"] == "Alter") & (df["Category"].str.contains(category, na=False, regex=False))
        ]
        if not row.empty:
            # **Absoluten Wert für jede Altersgruppe pro Studiengang berechnen**
            abs_bachelor = (row[bachelor_studiengaenge] / 100) * df[
                (df["Variable"] == "Alter") & (df["Category"].str.contains("Anzahl", na=False))
            ][bachelor_studiengaenge].iloc[0]

            abs_master = (row[master_studiengaenge] / 100) * df[
                (df["Variable"] == "Alter") & (df["Category"].str.contains("Anzahl", na=False))
            ][master_studiengaenge].iloc[0]

            # **Gesamte Anzahl an Studierenden in dieser Altersgruppe berechnen**
            total_abs_students = abs_bachelor.sum().sum() + abs_master.sum().sum()

            # **Gewichteter relativer Anteil berechnen (im Vergleich zur Gesamtzahl aller Studierenden)**
            weighted_total = (total_abs_students / total_students) * 100

            # Spaltennamen dynamisch hinzufügen
            col_name = f"{category}"  # Ohne Bachelor/Master Unterscheidung

            if col_name not in alter_data:
                alter_data[col_name] = []

            alter_data[col_name].append(weighted_total)

# Daten als DataFrame formatieren
df_alter = pd.DataFrame(alter_data).sort_values(by="Jahr")

In [68]:
import pandas as pd
import plotly.express as px

# Daten für die Alterspyramide umformen
df_pyramide = df_alter.melt(id_vars="Jahr", var_name="Altersgruppe", value_name="Anteil (%)")

# Bachelor vs. Master identifizieren
df_pyramide["Studiengang"] = df_pyramide["Altersgruppe"].apply(lambda x: "Bachelor" if "Bachelor" in x else "Master")

# Altersgruppen umbenennen (Bachelor/Master entfernen)
df_pyramide["Altersgruppe"] = df_pyramide["Altersgruppe"].str.replace("Bachelor ", "", regex=False)
df_pyramide["Altersgruppe"] = df_pyramide["Altersgruppe"].str.replace("Master ", "", regex=False)

# Bachelor-Werte negativ setzen für visuelle Trennung
df_pyramide.loc[df_pyramide["Studiengang"] == "Bachelor", "Anteil (%)"] *= -1

# Neuen DataFrame erstellen mit nur den letzten Jahrgang
jahrgang = "2013"  # Letztes verfügbares Jahr
df_pyramide_latest = df_pyramide[df_pyramide["Jahr"] == jahrgang]

In [69]:
jahrgang

'2013'

In [70]:
fig = px.bar(
    df_pyramide_latest, 
    x="Anteil (%)", 
    y="Altersgruppe", 
    color="Studiengang", 
    orientation="h",  # Horizontal
    title=f"Alterspyramide der Studierenden ({jahrgang})",
    color_discrete_map={"Bachelor": "#1f77b4", "Master": "#ff7f0e"},  # Blautöne für Bachelor, Orange für Master
)

# Layout-Anpassungen
fig.update_layout(
    #template="infoviz",
    xaxis_title="Anteil der Studierenden (%)",
    yaxis_title="Altersgruppe",
    xaxis=dict(
        tickmode="array",
        tickvals=[-20, -10, 0, 10, 20],  # Achse zentrieren
        ticktext=["20%", "10%", "0%", "10%", "20%"]
    ),
    legend_title="Studiengang"
)

fig.show()

In [71]:
# Neuen DataFrame erstellen mit nur den letzten Jahrgang
jahrgang = "2024"  # Letztes verfügbares Jahr
df_pyramide_latest = df_pyramide[df_pyramide["Jahr"] == jahrgang]

In [72]:
fig = px.bar(
    df_pyramide_latest, 
    x="Anteil (%)", 
    y="Altersgruppe", 
    color="Studiengang", 
    orientation="h",  # Horizontal
    title=f"Alterspyramide der Studierenden ({jahrgang})",
    color_discrete_map={"Bachelor": "#1f77b4", "Master": "#ff7f0e"},  # Blautöne für Bachelor, Orange für Master
)

# Layout-Anpassungen
fig.update_layout(
    #template="infoviz",
    xaxis_title="Anteil der Studierenden (%)",
    yaxis_title="Altersgruppe",
    xaxis=dict(
        tickmode="array",
        tickvals=[-20, -10, 0, 10, 20],  # Achse zentrieren
        ticktext=["20%", "10%", "0%", "10%", "20%"]
    ),
    legend_title="Studiengang"
)

fig.show()

In [57]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Daten für die Alterspyramide umformen
df_pyramide = df_alter.melt(id_vars="Jahr", var_name="Altersgruppe", value_name="Anteil (%)")

# Bachelor vs. Master identifizieren
df_pyramide["Studiengang"] = df_pyramide["Altersgruppe"].apply(lambda x: "Bachelor" if "Bachelor" in x else "Master")

# Altersgruppen umbenennen (Bachelor/Master entfernen)
df_pyramide["Altersgruppe"] = df_pyramide["Altersgruppe"].str.replace("Bachelor ", "", regex=False)
df_pyramide["Altersgruppe"] = df_pyramide["Altersgruppe"].str.replace("Master ", "", regex=False)

# Bachelor-Werte negativ setzen für visuelle Trennung
df_pyramide.loc[df_pyramide["Studiengang"] == "Bachelor", "Anteil (%)"] *= -1

# Einzigartige Jahre für Subplots
jahre = sorted(df_pyramide["Jahr"].unique(), reverse=True)
num_years = len(jahre)

# Subplots erstellen
fig = make_subplots(
    rows=num_years, cols=1,  # Eine Spalte, eine Zeile pro Jahr
    shared_xaxes=True,  # Gleiche X-Achse für bessere Vergleichbarkeit
    subplot_titles=[f"Alterspyramide {jahr}" for jahr in jahre]
)

# Farben definieren
farben = {"Bachelor": "#1f77b4", "Master": "#ff7f0e"}  # Blau für Bachelor, Orange für Master

# Daten für jedes Jahr als separaten Subplot hinzufügen
for i, jahr in enumerate(jahre, start=1):
    df_year = df_pyramide[df_pyramide["Jahr"] == jahr]

    for studiengang in ["Bachelor", "Master"]:
        df_group = df_year[df_year["Studiengang"] == studiengang]

        fig.add_trace(
            go.Bar(
                x=df_group["Anteil (%)"],
                y=df_group["Altersgruppe"],
                orientation="h",
                name=studiengang if i == 1 else None,  # Legende nur einmal anzeigen
                marker=dict(color=farben[studiengang]),
            ),
            row=i, col=1
        )

# Layout anpassen
fig.update_layout(
    title="Alterspyramide der Studierenden über die Jahre",
    #template="infoviz",
    xaxis_title="Anteil der Studierenden (%)",
    height=300 * num_years,  # Höhe dynamisch anpassen
    showlegend=True,
)

# X-Achse so formatieren, dass negative und positive Werte gleichmäßig verteilt sind
fig.update_xaxes(
    tickmode="array",
    tickvals=[-20, -10, 0, 10, 20],
    ticktext=["20%", "10%", "0%", "10%", "20%"]
)

fig.show()